# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

I suggest you implement *constraint satisfaction* as it is described in the unit material. You can use the code you have previously been given as a guide. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%).

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [4]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [186]:
import numpy as np
hard1 = np.array([ [0, 0, 0, 0, 0, 0, 6, 1, 9],
                   [0, 2, 3, 0, 9, 0, 0, 0, 0],
                   [0, 0, 0, 1, 4, 0, 0, 2, 0],
                   [0, 0, 9, 0, 0, 0, 0, 0, 0],
                   [7, 0, 8, 0, 0, 3, 0, 0, 0],
                   [0, 0, 0, 0, 0, 5, 3, 4, 0],
                   [0, 0, 0, 0, 0, 0, 4, 6, 7],
                   [8, 3, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, 6, 1, 2, 0, 0, 0]])

easy1 = np.array([ [0, 0, 5, 2, 6, 9, 7, 8, 1],
                   [0, 8, 2, 5, 7, 1, 4, 9, 3],
                   [1, 9, 7, 8, 3, 4, 5, 6, 2],
                   [8, 2, 6, 1, 9, 5, 3, 4, 7],
                   [3, 7, 4, 6, 8, 2, 9, 1, 5],
                   [9, 5, 1, 7, 4, 3, 6, 2, 8],
                   [5, 1, 9, 3, 2, 6, 8, 7, 4],
                   [2, 4, 8, 9, 5, 7, 1, 3, 6],
                   [7, 6, 3, 4, 1, 8, 2, 5, 9]])

impossible1 = np.array([ [7, 0, 0, 0, 5, 4, 0, 0, 6],
                         [5, 0, 0, 6, 0, 3, 0, 9, 7],
                         [0, 9, 0, 0, 0, 0, 0, 0, 0],
                         [9, 2, 0, 0, 0, 0, 0, 6, 0],
                         [1, 0, 0, 0, 9, 0, 0, 0, 3],
                         [0, 3, 0, 0, 0, 0, 0, 2, 1],
                         [0, 0, 0, 0, 0, 0, 0, 0, 0],
                         [0, 4, 0, 3, 0, 2, 5, 0, 0],
                         [2, 5, 0, 8, 6, 0, 0, 3, 4]])

wh =          np.array([ [8, 0, 0, 0, 0, 0, 0, 0, 0],
                         [0, 0, 3, 6, 0, 0, 0, 0, 0],
                         [0, 7, 0, 0, 9, 0, 2, 0, 0],
                         [0, 5, 0, 0, 0, 7, 0, 0, 0],
                         [0, 0, 0, 0, 4, 5, 7, 0, 0],
                         [0, 0, 0, 1, 0, 0, 0, 3, 0],
                         [0, 0, 1, 0, 0, 0, 0, 6, 8],
                         [0, 0, 8, 5, 0, 0, 0, 1, 0],
                         [0, 9, 0, 0, 0, 0, 4, 0, 0]])

easytest = np.array([[3, 7, 4, 5, 8, 0, 6, 2, 9],
            [8, 2, 5, 7, 9, 6, 4, 0, 1],
            [6, 9, 1, 2, 3, 4, 8, 7, 5],
            [1, 4, 2, 0, 5, 3, 7, 9, 8],
            [7, 8, 6, 9, 4, 2, 1, 5, 3],
            [9, 5, 3, 8, 1, 7, 2, 4, 6],
            [4, 6, 9, 1, 7, 5, 3, 8, 2],
            [5, 1, 7, 3, 2, 8, 0, 6, 4],
            [2, 3, 8, 6, 0, 9, 5, 1, 7]])

hard2 = np.array([[0, 2, 0, 0, 0, 6, 9, 0, 0],
         [0, 0, 0, 0, 5, 0, 0, 2, 0],
         [6, 0, 0, 3, 0, 0, 0, 0, 0],
         [9, 4, 0, 0, 0, 7, 0, 0, 0],
         [0, 0, 0, 4, 0, 0, 7, 0, 0],
         [0, 3, 0, 2, 0, 0, 0, 8, 0],
         [0, 0, 9, 0, 4, 0, 0, 0, 0],
         [3, 0, 0, 9, 0, 2, 0, 1, 7],
         [0, 0, 8, 0, 0, 0, 0, 0, 2]])

hardtest = np.array([[0, 0, 0, 0, 0, 7, 5, 4, 0],
 [9, 0, 6, 0, 5, 0, 0, 3, 0],
 [0, 0, 0, 0, 0, 0, 2, 0, 0],
 [2, 0, 0, 0, 0, 0, 7, 9, 0],
 [0, 0, 3, 0, 4, 1, 0, 0, 0],
 [7, 0, 0, 0, 0, 0, 0, 5, 0],
 [0, 3, 0, 0, 0, 4, 0, 2, 0],
 [0, 9, 4, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 5, 9, 0, 0, 0, 4]])

hardtest2 = np.array([[1, 0, 0, 7, 0, 0, 0, 0, 0],
             [0, 3, 2, 0, 0, 0, 0, 0, 0],
             [0, 0, 0, 6, 0, 0, 0, 0, 0],
             [0, 8, 0, 0, 0, 2, 0, 7, 0],
             [5, 0, 7, 0, 0, 1, 0, 0, 0],
             [0, 0, 0, 0, 0, 3, 6, 1, 0],
             [7, 0, 0, 0, 0, 0, 2, 0, 9],
             [0, 0, 0, 0, 5, 0, 0, 0, 0],
             [3, 0, 0, 0, 0, 4, 0, 0, 5]])

In [376]:
import copy

class board():
    
    def __init__(self, board):
        """ Creates board class with attributes final / possibilities"""
        self.start = board.flatten()
        self.final = np.array([-1 if i == 0 else i for i in np.nditer(self.start)])
        
        self.possibilities = np.array([range(1,10) if _ == 0 else [0] * 9 for _ in np.nditer(self.start)])
        for i,v in np.ndenumerate(self.start):
            if v != 0:
                self.possibilities[i[0]][v - 1] = v
            else:
                cells = self.get_col_row_grid(i[0])
                for idx in set(cells[0] + cells[1] + cells[2]):
                    if self.final[idx] != -1:
                        self.possibilities[i[0]][self.final[idx]-1] = 0
                        
    @staticmethod
    def get_col_row_grid(position):
        """ Returns the column / row / grid indices for a given position """
        col_idx = [(9*_)+ position%9 for _ in range(9)]
        row_idx = list(range(position-(position%9),position-(position%9)+9))
        col_pos = position%9 - position%9%3
        row_pos = ((position//9)*9) - ((position//9)*9)%27
        grid_idx = [col_pos + row_pos + _ for _ in range(3)] + \
                    [col_pos + row_pos + _ for _ in range(9,12)] + \
                    [col_pos + row_pos + _ for _ in range(18,21)]
        return col_idx, row_idx, grid_idx
    
    def get_unique_value_count(self):
        unique, counts = np.unique(self.final, return_counts = True)
        return dict(zip(unique, counts))
    
    def naked_pair(self, positions): 
        naked_pairs = []
        for pos1 in positions:
            for pos2 in positions:
                if pos1 != pos2:
                    if self.final[pos1] == -1 and self.final[pos2] == -1:
                        if len(self.get_possible_values(pos1)) == 2 and len(self.get_possible_values(pos1)) == 2:
                            if all(self.possibilities[pos1] == self.possibilities[pos2]):
                                if (pos1, pos2) not in naked_pairs and (pos2, pos1) not in naked_pairs:
                                    naked_pairs.append((pos1, pos2))
                                    
        return naked_pairs
                            
    def is_valid_answer(self):
        """ Returns True if sudoku with 1-9 in rows, 1-9 in columns and 1-9 in boxes """
        rows = [set([self.final[i+(9*j)] for i in range(0,9)]) == set(range(1,10)) for j in range(0,9)]
        cols = [set([self.final[(9*i)+j] for i in range(0,9)]) == set(range(1,10)) for j in range(0,9)]
        grid = [set([self.final[col + row + _] for _ in range(3)] +
                 [self.final[col + row + _] for _ in range(9,12)] +
                 [self.final[col + row + _] for _ in range(18,21)]) == set(range(1,10)) for col in [0,3,6] for row in [0, 27, 54]]

        return self.final.reshape(9,9) if all(rows) == True and all(cols) == True and all(grid) == True else np.full((9,9),-1)
     
    def is_goal(self):
        """ Returns True if all values in the final array are not -1 """
        return True if np.all(self.final != -1) else False
      
    def is_invalid(self):
        """ Returns False if there is an array in possibilities full of 0's """
        return True if np.any([not np.any(i) for i in self.possibilities]) else False
    
    def get_singletons(self):
        """ Returns the positions where we have a 'certain' value but final not updated """
        return [index[0] for index,n in np.ndenumerate(np.count_nonzero(self.possibilities == 0, axis = 1)) if (self.final[index] == -1 and n == 8)]
        
    def get_possible_values(self, position):
        """ Returns the possible value from a given position (excludes 0) """
        arr = self.possibilities[position]
        return arr[np.where(arr != 0)].copy()
    
    def set_value(self, position, value): #position(0-80)
        """ Creates new state, adds in position, checks if further improvements can be made """

        state = copy.deepcopy(self)
        state.possibilities[position] = [0 if i != value else value for i in range(1,10)]
        state.final[position] = value

        # REMOVE FROM ROW / COLUMNS / GRID POSSIBILITIES
        col, row, grid = self.get_col_row_grid(position)
        for pos in set(col + row + grid):
            if pos != position:
                if self.final[pos] == -1:
                    state.possibilities[pos][value - 1] = 0
           
        # APPLY NAKED PAIRS ON ROW AND COLUMN
        col_naked_pair = self.naked_pair(col)
        if col_naked_pair:
            for pair in col_naked_pair:
                pair_positions = self.get_possible_values(pair[0])
                for pos in col:
                    if self.final[pos] == -1 and (pos != pair[0] or pos != pair[1]):
                        if self.get_possible_values(pair[0]) != []:
                        #print(pair, self.get_possible_values(pair[0]), " pos ", pos, "Pair pos", pair_positions[0], pair_positions[1])
                            self.possibilities[pos][pair_positions[0] - 1] = 0
                            self.possibilities[pos][pair_positions[1] - 1] = 0
                            
        row_naked_pair = self.naked_pair(row)
        if row_naked_pair:
            for pair in row_naked_pair:
                pair_positions = self.get_possible_values(pair[0])
                for pos in row:
                    if self.final[pos] == -1 and (pos != pair[0] or pos != pair[1]):
                        if self.get_possible_values(pair[0]) != []:
                            self.possibilities[pos][pair_positions[0] - 1] = 0
                            self.possibilities[pos][pair_positions[1] - 1] = 0

        return self.set_value_loop(state)
    
    def set_value_loop(self, state):
        """ Recrusive loop to turn all singletons into final values (and then reset)"""
        singleton_positions = state.get_singletons()
        while len(singleton_positions) > 0:
            new_pos = singleton_positions[0]
            state = state.set_value(new_pos, state.get_possible_values(new_pos)[0])
            singleton_positions = state.get_singletons()
        
        return state
    

In [342]:
x = board(easy1)
x.get_possible_values(1)

array([3])

In [299]:
x = np.array([0,1,2])
y = np.array([0,1,2])
all(x == y) 

True

In [291]:
def pick_next_position(partial_state):
    """ Extracts the positions where more than 1 non-zero value, selects a position randomly """
    pos_idx = [i for i in range(len(partial_state.possibilities)) if np.count_nonzero(partial_state.possibilities[i]) > 1]
    return pos_idx

def order_values(partial_state, position):
    """ Returns the possible values to try for a given position and state """
    values = partial_state.get_possible_values(position)
    return values
 
def depth_first_search(partial_state):
    """ Perform depth first search, starting with least possibles / lowest values (to force answer)"""
    
    position = pick_next_position(partial_state)
    
    if position == []:
        return partial_state
    else:
        position = np.sort(position)[0]
    
    
    values = order_values(partial_state, position)
    common_values = partial_state.get_unique_value_count()
    ordered_values = []
    for temp_value in values:
        ordered_values.append((temp_value, common_values.get(temp_value,0)))
    ordered_values.sort(key = lambda x: x[1], reverse = True)
        
    for value in ordered_values:
        new_state = partial_state.set_value(position, value[0])
        if new_state.is_goal():
            return new_state
        if not new_state.is_invalid():
            deep_state = depth_first_search(new_state)
            if deep_state is not None and deep_state.is_goal():
                return deep_state
    return None
    
def sudoku_solver(sudoku):
    
    partial_state = board(sudoku)

    singleton_positions = partial_state.get_singletons()
    while len(singleton_positions) > 0:
        new_pos = singleton_positions[0]
        partial_state = partial_state.set_value(new_pos, partial_state.get_possible_values(new_pos)[0])
        singleton_positions = partial_state.get_singletons()
    
    if partial_state.is_goal():
        return partial_state.is_valid_answer()
    
    answer = depth_first_search(partial_state)
    return np.full((9,9),-1) if answer == None else answer.is_valid_answer()


In [292]:
values = [4,7,3]
common_values = {-1:20, 4:6, 3:2, 9:4}
ordered_values = []
for temp_value in values:
    ordered_values.append((temp_value, common_values.get(temp_value,0)))
ordered_values.sort(key = lambda x: x[1], reverse = True)
ordered_values


[(4, 6), (3, 2), (7, 0)]

In [ ]:
sudoku_solver(hardtest)

In [344]:
sudoku_solver(easy1)

array([[4, 3, 5, 2, 6, 9, 7, 8, 1],
       [6, 8, 2, 5, 7, 1, 4, 9, 3],
       [1, 9, 7, 8, 3, 4, 5, 6, 2],
       [8, 2, 6, 1, 9, 5, 3, 4, 7],
       [3, 7, 4, 6, 8, 2, 9, 1, 5],
       [9, 5, 1, 7, 4, 3, 6, 2, 8],
       [5, 1, 9, 3, 2, 6, 8, 7, 4],
       [2, 4, 8, 9, 5, 7, 1, 3, 6],
       [7, 6, 3, 4, 1, 8, 2, 5, 9]])

In [381]:
sudoku_solver(hard2)

<ipython-input-376-e9af904ffd4f>:108: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if self.get_possible_values(pair[0]) != []:
<ipython-input-376-e9af904ffd4f>:108: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if self.get_possible_values(pair[0]) != []:
<ipython-input-376-e9af904ffd4f>:97: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if self.get_possible_values(pair[0]) != []:
<ipython-input-376-e9af904ffd4f>:97: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if self.get_possible_values(pair[0]) != []:


array([[-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [379]:
sudoku_solver(easy1)

array([[4, 3, 5, 2, 6, 9, 7, 8, 1],
       [6, 8, 2, 5, 7, 1, 4, 9, 3],
       [1, 9, 7, 8, 3, 4, 5, 6, 2],
       [8, 2, 6, 1, 9, 5, 3, 4, 7],
       [3, 7, 4, 6, 8, 2, 9, 1, 5],
       [9, 5, 1, 7, 4, 3, 6, 2, 8],
       [5, 1, 9, 3, 2, 6, 8, 7, 4],
       [2, 4, 8, 9, 5, 7, 1, 3, 6],
       [7, 6, 3, 4, 1, 8, 2, 5, 9]])

In [380]:
sudoku_solver(wh)


<ipython-input-376-e9af904ffd4f>:108: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if self.get_possible_values(pair[0]) != []:
<ipython-input-376-e9af904ffd4f>:108: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if self.get_possible_values(pair[0]) != []:
<ipython-input-376-e9af904ffd4f>:97: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if self.get_possible_values(pair[0]) != []:
<ipython-input-376-e9af904ffd4f>:97: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if self.get_possible_values(pair[0]) != []:


array([[8, 1, 2, 7, 5, 3, 6, 4, 9],
       [9, 4, 3, 6, 8, 2, 1, 7, 5],
       [6, 7, 5, 4, 9, 1, 2, 8, 3],
       [1, 5, 4, 2, 3, 7, 8, 9, 6],
       [3, 6, 9, 8, 4, 5, 7, 2, 1],
       [2, 8, 7, 1, 6, 9, 5, 3, 4],
       [5, 2, 1, 9, 7, 4, 3, 6, 8],
       [4, 3, 8, 5, 2, 6, 9, 1, 7],
       [7, 9, 6, 3, 1, 8, 4, 5, 2]])

In [937]:
sudoku_valid_answer(ans)

'OK'

In [ ]:
def sudoku_solver(sudoku):
    
    def depth_first_search():
    
    return solved_sudoku

All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [382]:
SKIP_TESTS = False

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.0 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5

<ipython-input-376-e9af904ffd4f>:97: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if self.get_possible_values(pair[0]) != []:
<ipython-input-376-e9af904ffd4f>:97: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if self.get_possible_values(pair[0]) != []:


This is your solution for medium sudoku number 4
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.015625 seconds to solve.

This is medium sudoku number 5
[[0 8 5 0 1 3 0 0 9]
 [6 3 4 0 0 2 1 7 5]
 [0 2 0 5 7 4 0 3 0]
 [2 4 8 3 6 7 9 5 1]
 [9 6 0 4 5 8 0 2 3]
 [3 5 7 2 0 0 4 8 0]
 [5 7 3 1 0 0 8 9 2]
 [4 9 6 0 2 5 3 1 0]
 [8 1 2 0 3 9 5 6 4]]
This is your solution for medium sudoku number 5
[[7 8 5 6 1 3 2 4 9]
 [6 3 4 9 8 2 1 7 5]
 [1 2 9 5 7 4 6 3 8]
 [2 4 8 3 6 7 9 5 1]
 [9 6 1 4 5 8 7 2 3]
 [3 5 7 2 9 1 4 8 6]
 [5 7 3 1 4 6 8 9 2]
 [4 9 6 8 2 5 3 1 7]
 [8 1 2 7 3 9 5 6 4]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.015625 seconds to solve.

This is medium sudoku number 6
[

<ipython-input-376-e9af904ffd4f>:108: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if self.get_possible_values(pair[0]) != []:
<ipython-input-376-e9af904ffd4f>:108: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if self.get_possible_values(pair[0]) != []:


This is your solution for hard sudoku number 0
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 3.265625 seconds to solve.

This is hard sudoku number 1
[[1 0 0 7 0 0 0 0 0]
 [0 3 2 0 0 0 0 0 0]
 [0 0 0 6 0 0 0 0 0]
 [0 8 0 0 0 2 0 7 0]
 [5 0 7 0 0 1 0 0 0]
 [0 0 0 0 0 3 6 1 0]
 [7 0 0 0 0 0 2 0 9]
 [0 0 0 0 5 0 0 0 0]
 [3 0 0 0 0 4 0 0 5]]
This is your solution for hard sudoku number 1
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
Th

KeyboardInterrupt: 

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [943]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p1 = pathlib.Path('./readme.txt')
p2 = pathlib.Path('./readme.md')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("You must include a separate file called readme.txt or readme.md in your submission.")
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named sudoku.ipynb")
    
if "sudoku_solver" not in dir():
    fail = True;
    print("You must include a function called sudoku_solver which accepts a numpy array.")
else: 
    sudoku = np.load("data/very_easy_puzzle.npy")[0]
    solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(sudoku), solution):
        print("Warning:")
        print("Your sudoku_solver function does not correctly solve the first sudoku.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [ ]:
# This is a TEST CELL. Do not delete or change.